In [1]:
import numpy as np
from BPmodule.BPGen  import PROBLEM2,PROBLEM3,PROBLEM4,PROBLEM5
from BPnumba.NumFun import CreatePoblation
from BPnumba.NumAG import create_intidivual,Ind,CalcFi
from BPnumba.DFFAnumba import DFFtrain
from numba.typed import List as NumbaList
from numba.typed import Dict
from numba import njit,prange,objmode
from typing import List
import time


In [2]:
P2A1 = PROBLEM2[0]
P2A2 = PROBLEM2[1]
P3A1 = PROBLEM3[0]
P3A2 = PROBLEM3[1]
P4A1 = PROBLEM4[0]
P4A2 = PROBLEM4[1]
P5A1 = PROBLEM5[0]
P5A2 = PROBLEM5[1]
gamma = 0.004
contenedor = np.asanyarray(P3A1[0],dtype=np.int64)
datos =np.asanyarray(P3A1[1],dtype=np.int64)
pob = CreatePoblation(100,np.arange(1,27))



In [3]:
from BPnumba.IndPqueue import CreateGenPriorityQueue,GPQueue
pobQ = GPQueue(50)

In [4]:
for i in range(100):
    ind1:Ind = create_intidivual(NumbaList(pob[i]))
    CalcFi(ind1,datos,contenedor)
    print(ind1.fi)
    pobQ.push(ind1)


In [5]:
pobQ.Print()

0.815244 [6, 20, 17, 19, 2, 3, 23, 26, 18, 25, 21, 13, 4, 9, 5, 12, 15, 11, 1, 16, 22, 7, 10, 8, 24, 14, ...]
0.815244 [4, 18, 14, 24, 6, 19, 2, 23, 20, 25, 11, 10, 8, 17, 3, 21, 15, 16, 13, 1, 7, 9, 26, 22, 12, 5, ...]
0.815244 [8, 10, 14, 2, 5, 15, 22, 23, 26, 12, 20, 19, 6, 17, 24, 1, 16, 7, 13, 4, 3, 9, 11, 25, 18, 21, ...]
0.815244 [1, 19, 23, 26, 6, 10, 21, 12, 25, 7, 4, 3, 22, 8, 9, 18, 11, 5, 20, 14, 13, 2, 17, 16, 15, 24, ...]
0.815244 [24, 6, 13, 22, 2, 5, 25, 23, 12, 18, 16, 17, 11, 21, 19, 3, 4, 8, 10, 9, 20, 7, 14, 26, 15, 1, ...]
0.815244 [26, 23, 15, 11, 24, 12, 20, 3, 18, 19, 7, 2, 21, 9, 16, 25, 8, 6, 22, 14, 17, 10, 4, 1, 5, 13, ...]
0.815244 [26, 25, 22, 17, 19, 18, 2, 20, 24, 9, 14, 16, 8, 4, 6, 15, 1, 23, 10, 5, 11, 3, 21, 13, 12, 7, ...]
0.815244 [6, 12, 8, 18, 11, 5, 1, 3, 23, 21, 13, 22, 9, 4, 26, 20, 2, 14, 25, 10, 15, 19, 7, 16, 24, 17, ...]
0.778092 [18, 26, 20, 2, 23, 7, 10, 4, 15, 3, 13, 25, 14, 17, 19, 12, 5, 11, 6, 9, 8, 21, 1, 22, 16, 24, ...]
0.778092 [